In [1]:
import numpy as np
import pandas as pd

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [22]:
data = pd.read_csv('data/Project 1 data.csv')
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 0:1024]
Y = Y[:, 1024]

#Split to train/test
X_train = X[0:70000, :]
X_train = X_train/255.  #Value of pixels range from 0-255. So change it from 0-1 (Normalisation basically)
X_test = X[70000:72001, :]
X_test = X_test/255.

#Reshape
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:70000, :]
Y_train =Y_train.T
Y_test = Y[70000:72001, :]
Y_test = Y_test.T

In [23]:
print("Number of training examples = ", str(X_train.shape[0]))
print("Number of test examples = ", str(X_test.shape[0]))
print("X_train shape: ", str(X_train.shape))
print("Y_train shape: ", str(Y_train.shape))
print("X_test shape: ", str(X_test.shape))
print("Y_test shape: ", str(Y_test.shape))

Number of training examples =  70000
Number of test examples =  2000
X_train shape:  (70000, 1024)
Y_train shape:  (1, 70000)
X_test shape:  (2000, 1024)
Y_test shape:  (1, 2000)


In [24]:
#Define the height and width of images
image_x = 32
image_y = 32

#keras function - basically used for one-hot encoding
train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)

#Shape is basically number of rows across number of classes now (Number of digits/alpha to identify)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])

#The 1 here refers to number of channels. If image is Black&White then channels is 1. If RGB, then channels is 3
X_train = X_train.reshape(X_train.shape[0], image_x, image_y, 1)
X_test = X_test.reshape(X_test.shape[0], image_x, image_y, 1)

In [25]:
print("X_train shape: ", str(X_train.shape))
print("Y_train shape: ", str(train_y.shape))

X_train shape:  (70000, 32, 32, 1)
Y_train shape:  (70000, 37)


### Build model now

In [26]:
def keras_model(image_x, image_y): #ht, width
    num_of_classes = 37
    #Sequential model. Can use others also
    model = Sequential() 
    
    #apply a conv filter. 2D convolution layer (e.g. spatial convolution over images).
    #filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
    #kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.
    model.add(Conv2D(filters=32, kernel_size=(5,5), input_shape=(image_x, image_y, 1), activation='relu'))
    
    #Max pooling operation for spatial data.
    #pool_size: integer or tuple of 2 integers, factors by which to downscale (vertical, horizontal). (2, 2) will halve the input in both spatial dimension
    #padding: One of "valid" or "same" (case-insensitive).
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    
    #Only the first layer in a model needs to know input_size. Others can infer
    model.add(Conv2D(filters=64, kernel_size=(5,5), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    
    #Flattening a tensor means to remove all of the dimensions except for one. This is exactly what the Flatten layer do.
    model.add(Flatten())
    
    #Last layer. Used to output a Y value. Softmax classifier
    model.add(Dense(num_of_classes, activation='softmax'))
    
    #Compile defines the loss function, the optimizer and the metrics. That's all
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    filepath = 'model/devanagari_model_refined.h5'
    
    #The ModelCheckpoint callback class allows you to define where to checkpoint the model weights, how the file should named and under what circumstances to make a checkpoint of the model.
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]
    
    return model, callbacks_list

In [28]:
model, callbacks_list = keras_model(image_x, image_y)
# Fit model. Will try to minimise loss. 64 batch_size to perform gradient descent. Then goes to perform for the next 64 bytes
# Can use any number of epochs. We'll use 1 now just for testing purposes
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=1, batch_size=64, callbacks=callbacks_list)
scores = model.evaluate(X_test, test_y, verbose=0)
print("CNN Error: ", (100-scores[1]*100))

#Exactly what all functionality we're using
print_summary(model)
model.save('model/devanagari.h5')

Train on 70000 samples, validate on 2000 samples
Epoch 1/1
70000/70000 [==============================] - 81s 1ms/step - loss: 1.3635 - acc: 0.6902 - val_loss: 0.5293 - val_acc: 0.8765

Epoch 00001: val_acc improved from -inf to 0.87650, saving model to model/devanagari_model_refined.h5
CNN Error:  12.350000000000009
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)  